In [ ]:
import numpy as np
from scipy.spatial import ConvexHull
import trimesh
from matplotlib import pyplot as plt
from typing import List, Dict
import itertools

Проверка наличия блендера на устройстве, чтобы использовать его движок в дальнейшем.

In [ ]:
print(trimesh.interfaces.blender.exists)
engine = 'blender'


Загрузка моделей, где cube - это предпологаемая зона интереса(ЗИ), а ball - сфера, которая будет покрывать зону.

In [ ]:
ball = trimesh.load_mesh('resources/ball.stl')
cube = trimesh.load_mesh('resources/cube.stl')
ball = ball.apply_scale(5)
# cube = cube.apply_scale(0.5)

In [ ]:
def ball_radius(ball: trimesh.Trimesh) -> float:
    vertexes = trimesh.bounds.corners(ball.bounds)
    source = np.array(ball.center_mass)
    dest = np.array(vertexes[0])
    vec = dest - source
    return np.linalg.norm(vec) * (2 ** 0.5) / 2 / 2

In [ ]:
def print_balls(balls: List[trimesh.Trimesh]) -> trimesh.scene.scene.Scene:
    scene = trimesh.Scene()
    for item in balls:
        scene.add_geometry(item)
    return scene

In [ ]:
r = ball_radius(ball)
def translation(i, j, k) -> np.array:
    """
    return translation for ijk hcp sphere's pos
    """
    return np.array([2*i + ((j+k) % 2), (j + ((3**0.5)*(k % 2)/3)), 2 / 3 * (6 ** 0.5) * k])


def check_dict(balls_dict, x, y, z) -> str:
    if balls_dict.get(z):
        if balls_dict.get(z).get(y):
            if balls_dict.get(z).get(y).get(x):
                return "exist"
            else:
                return "x"
        else:
            return "y"
    return "z"


def fill_check(balls_dict: Dict[int, Dict[int, Dict[int, trimesh.Trimesh]]], x, y, z, tumor: trimesh.Trimesh, queue: List[np.ndarray]) -> None:
    check = check_dict(balls_dict, x, y, z)
    if check == "exist":
        return
    new_ball = ball.copy().apply_transform(trimesh.transformations.translation_matrix(translation(x, y, z) * r))
    temp = list(new_ball.vertices)
    temp.append(list(new_ball.center_mass))
    if tumor.contains(temp).any():
        queue.append(np.array([x + 1, y + 1, z + 1]))
        queue.append(np.array([x - 1, y - 1, z - 1]))
        queue.append(np.array([x + 1, y, z]))
        queue.append(np.array([x, y + 1, z]))
        queue.append(np.array([x, y, z + 1]))
        queue.append(np.array([x - 1, y, z]))
        queue.append(np.array([x, y - 1, z]))
        queue.append(np.array([x, y, z - 1]))
        queue.append(np.array([x + 1, y, z + 1]))
        queue.append(np.array([x, y + 1, z + 1]))
        queue.append(np.array([x + 1, y + 1, z]))
        queue.append(np.array([x - 1, y, z - 1]))
        queue.append(np.array([x, y - 1, z - 1]))
        queue.append(np.array([x - 1, y - 1, z]))
    else:
        return
    if check == "x":
        balls_dict[z][y][x] = new_ball
    elif check == "y":
        balls_dict[z][y] = {x: new_ball}
    elif check == "z":
        balls_dict[z] = {y: {x: new_ball}}
    return    


def fill_hcp(tumor: trimesh.Trimesh, ball: trimesh.Trimesh) -> List[trimesh.Trimesh]:
    balls_dict = {}
    balls = []
    queue = []
    queue.append(np.array([0, 0, 0]))
    while(queue):
        print("number of points in queue:", len(queue))
        item = queue.pop()
        x = item[0]
        y = item[1]
        z = item[2]
        fill_check(balls_dict, x , y , z , tumor, queue=queue)
    for zV in balls_dict.values():
        for yV in zV.values():
            for xV in yV.values():
                balls.append(xV)
    return balls

Заливка опухоли.

In [ ]:
b = fill_hcp(cube, ball)

In [ ]:
print_balls(b).show()

In [ ]:
buff = [cube]
buff.extend(b)
diff = trimesh.boolean.difference(buff, engine='blender')
print("Результат операции пустота? -", type(diff) == trimesh.scene.scene.Scene)
diff.show()

Смещение сфер к центру, пока опухоль не выйдет за пределы выпуклой оболочки перекрытия сферами.

In [ ]:
lr = 0.005
b.reverse()
i = 0
contain = True
while(contain):
    i += 1
    print(i, b[0].center_mass)
    for ball in b[:-1]:
        vec = b[-1].center_mass - ball.center_mass
        pos = ball.center_mass
        translate = (lr * vec * r / np.linalg.norm(vec))
        ball.apply_transform(trimesh.transformations.translation_matrix(translate))
    hull = trimesh.points.PointCloud(list(itertools.chain.from_iterable([list(elem.vertices) for elem in b]))).convex_hull
    print(hull.vertices.shape)
    #diff = trimesh.boolean.difference(buff, engine='blender')
    contain = hull.contains(cube.vertices).all()

In [ ]:
print_balls(b).show()

In [ ]:
buff = [cube]
buff.extend(b)
diff = trimesh.boolean.difference(buff, engine='blender')
print("Результат операции пустота? -", type(diff) == trimesh.scene.scene.Scene)
diff.show()

In [ ]:
diff.volume